# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Create-the-units" data-toc-modified-id="Create-the-units-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Create the units</a></div><div class="lev1 toc-item"><a href="#Run" data-toc-modified-id="Run-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Run</a></div>

This tutorial requires 
- pyPitchTracking https://github.com/brunodigiorgi/pyAudioGraph
- pyPitchTracking https://github.com/brunodigiorgi/pyPitchTracking

In [1]:
import pyAudioGraph as ag
import pyPitchTracking as pypt

# Create the units

In [2]:
nchannels = 2
sample_rate = 44100
input_mode = "disk"  # choose "mic" or "disk" 

# parameters
frame_size = 2048
hop_size = 1024
pri_stab = 450
pri_vtp = -.001
max_freq = 1500
min_weight = 0.4

# world
w = ag.World(buf_len=hop_size, sample_rate=sample_rate)

if(input_mode == "disk"):
    wav_file_path = '../../PitchTracking/Dataset/AdeleHello.wav'  # set a *monophonic* audio file path
    astream = ag.AudioStreamWaveFile(wav_file_path)
    assert((astream.nchannels == nchannels) and (astream.sample_rate == sample_rate))
    in_node = ag.Nodes.DiskInNode(w, astream)
elif(input_mode == "mic"):
    in_node = ag.Nodes.InNode(w)

# pitch tracking
context_movingviterbi = pypt.PitchTrackerContext.PitchTrackerProbContextMovingViterbi(pri_stab=pri_stab, pri_vtp=pri_vtp)
pitch_tracker = pypt.PitchTrackerYin.PitchTrackerPYin(sample_rate=sample_rate, 
                                                      frame_size=frame_size, 
                                                      hop_size=hop_size,
                                                      max_freq=max_freq, 
                                                      min_weight=min_weight, 
                                                      context=context_movingviterbi)
pitch = pypt.PitchTrackerNode.PitchTrackerNode(w, pitch_tracker)

# control rate recorder, with 2 tracks
rec = ag.Nodes.ControlRateRecorder(w, 2)

# rmsUnit
rmsUnit = ag.Nodes.RmsNode(w)

osc = ag.Nodes.SawOsc(w)
fil_osc = ag.Nodes.Lowpass(w, f0=1600, Q=1)
osc.w_out.plug_into(fil_osc.w_in)

out = ag.Nodes.OutNode(w)

# stereo to mono
stm = ag.Nodes.MonizerNode(w, in_node)
stm.w_out[0].plug_into(pitch.w_in)

# drive sin osc with rms level from input
stm.w_out[0].plug_into(rmsUnit.w_in)

sin_osc_level = ag.Nodes.AudioSlopeGen(w, initial_value=1, speed=.9)
rmsUnit.w_out.plug_into(sin_osc_level.w_in)
sin_mult = ag.Nodes.AudioOpMult(w)
fil_osc.w_out.plug_into(sin_mult.w_in1)
sin_osc_level.w_out.plug_into(sin_mult.w_in2)

# record pitch
pitch.w_f0.plug_into(rec.w_in[0])
pitch.w_voiced.plug_into(rec.w_in[1])

# reproduce pitch
pitch.w_f0.plug_into(osc.w_freq)
pitch.w_f0.plug_into(fil_osc.w_f0)

# level slope for smoothing voiced - non voiced transitions
levelSlopeUnit = ag.Nodes.AudioSlopeGen(w, initial_value=0, speed=.5)
pitch.w_voiced.plug_into(levelSlopeUnit.w_in)
mult = ag.Nodes.AudioOpMult(w)
sin_mult.w_out.plug_into(mult.w_in1)
levelSlopeUnit.w_out.plug_into(mult.w_in2)

# mix input and sin
v_sin, v_input = 2, 0.15
mix = ag.Nodes.MixerNode(w, np.array([[v_sin, 0], [v_sin, v_input]]))
mult.w_out.plug_into(mix.w_in[0])
stm.w_out[0].plug_into(mix.w_in[1])
mix.w_out[0].plug_into(out.w_in[0])
mix.w_out[1].plug_into(out.w_in[1])

# add output nodes and compile the graph
w.append([out, rec])
w.sort()

# Run 

In [3]:
import time

if(input_mode == "disk"):
    in_node.seek(20 * 44100)
    in_node.prime()
pitch.clear()
rec.clear()

w.start()
time.sleep(20)
w.stop()